In [2]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from requests.exceptions import HTTPError
import json
spark = SparkSession.builder.master('local[*]').appName('Capstone_EDA').getOrCreate()

OpenSea NFT Data

In [3]:
df = spark.read.json('/home/roger/SB/Capstone/Open_capstone_git/data/CryptoPunks')         
df.show()  

+----------------------+-------------+--------------------+----------------+--------------------+--------------------+--------+-----------+--------------------+------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+----------------+---------+--------------------+--------------------+-----------+--------+--------------+-------+--------------------+------------+--------------------------+
|animation_original_url|animation_url|      asset_contract|background_color|          collection|             creator|decimals|description|       external_link|    id|  image_original_url|   image_preview_url| image_thumbnail_url|           image_url|is_presale|           last_sale|listing_date|            name|num_sales|               owner|           permalink|sell_orders|token_id|token_metadata|top_bid|              traits|transfer_fee|transfer_fee_payment_token|
+----------------------+-------------+--------------------

In [4]:

df = spark.read.json('/home/roger/SB/Capstone/Open_capstone_git/data/CryptoPunks')           
df1=df.select(
    df['asset_contract']['name'].alias('NFT'),\
    df['token_id'],\
    df['num_sales'],\
    df['owner']['user']['username'].alias('username'),\
    df['owner']['address'].alias('owner_address'),\
    to_timestamp(df['last_sale']['event_timestamp']).alias('txn_date'),\
    (df['last_sale']['total_price']/10**18).alias('payment_amt'),\
    df['last_sale']['payment_token']['symbol'].alias('payment_type'))

df1.show()


+-----------+--------+---------+--------------------+--------------------+-------------------+------------------+------------+
|        NFT|token_id|num_sales|            username|       owner_address|           txn_date|       payment_amt|payment_type|
+-----------+--------+---------+--------------------+--------------------+-------------------+------------------+------------+
|CryptoPunks|    9998|        4|0xef764bac8a438e7...|0xef764bac8a438e7...|2021-10-28 23:50:37|124457.06752488602|         ETH|
|CryptoPunks|    7804|        2|                null|0xf4b4a58974524e1...|2021-03-11 00:48:39|            4200.0|         ETH|
|CryptoPunks|    3100|        2|                null|0x7b8961f67382c51...|2021-03-11 20:57:01|            4200.0|         ETH|
|CryptoPunks|    4156|        5|         TheReal4156|0x4c1194386e12156...|2021-12-09 20:38:17|            2500.0|         ETH|
|CryptoPunks|    5217|        2|-----------------...|0xe301612b0751f6d...|2021-07-30 21:24:47|            2250.

In [5]:
df1.printSchema()

root
 |-- NFT: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- num_sales: long (nullable = true)
 |-- username: string (nullable = true)
 |-- owner_address: string (nullable = true)
 |-- txn_date: timestamp (nullable = true)
 |-- payment_amt: double (nullable = true)
 |-- payment_type: string (nullable = true)



CryptoCompare Historical USD Data

In [6]:
# converting to UNIX timestamp in order to make API calls for USD historical price
unix_df=df1.select(df1['txn_date']).withColumn('unix',unix_timestamp(df1['txn_date']))
unix_df.show()

+-------------------+----------+
|           txn_date|      unix|
+-------------------+----------+
|2021-10-28 23:50:37|1635490237|
|2021-03-11 00:48:39|1615452519|
|2021-03-11 20:57:01|1615525021|
|2021-12-09 20:38:17|1639111097|
|2021-07-30 21:24:47|1627705487|
|2021-09-11 18:41:23|1631410883|
|2021-07-30 18:36:56|1627695416|
|2021-08-24 06:34:02|1629812042|
|2021-08-06 18:18:56|1628299136|
|2021-09-04 21:39:01|1630816741|
|2021-08-28 14:24:55|1630185895|
|2021-07-30 23:45:46|1627713946|
|2021-07-31 14:11:48|1627765908|
|2021-02-19 14:27:08|1613773628|
|2021-07-30 23:46:38|1627713998|
|2021-04-27 15:44:22|1619563462|
|2021-01-23 19:12:33|1611457953|
|2021-02-22 01:01:30|1613984490|
|2021-07-05 22:49:37|1625550577|
|2021-05-02 06:39:23|1619962763|
+-------------------+----------+
only showing top 20 rows



In [7]:
#creating UDF to that 
def usd_convert(unix):
    cryptocompare_url=f'https://min-api.cryptocompare.com/data/pricehistorical?fsym=ETH&tsyms=USD&ts={unix}'
    cryptocompare_api_key='0322f4e8b3f656c131992c34835efabf293037a7fdabb0e1aa4cf3664a3ffaa8'
    response=requests.get(cryptocompare_url,params={'api_key':cryptocompare_api_key})
    response_dic=response.json()
    return response_dic['ETH']['USD']

dollar_udf=udf(lambda x : usd_convert(x))
dollar_df=unix_df.withColumn('usd_rate',dollar_udf(unix_df['unix'])).drop(unix_df['unix']).cache()
dollar_df.show()

+-------------------+--------+
|           txn_date|usd_rate|
+-------------------+--------+
|2021-10-28 23:50:37|  4417.4|
|2021-03-11 00:48:39| 1827.06|
|2021-03-11 20:57:01| 1767.77|
|2021-12-09 20:38:17| 3902.49|
|2021-07-30 21:24:47| 2531.75|
|2021-09-11 18:41:23| 3405.32|
|2021-07-30 18:36:56| 2531.75|
|2021-08-24 06:34:02| 3172.57|
|2021-08-06 18:18:56| 3162.43|
|2021-09-04 21:39:01| 3952.33|
|2021-08-28 14:24:55| 3246.78|
|2021-07-30 23:45:46| 2531.75|
|2021-07-31 14:11:48| 2531.75|
|2021-02-19 14:27:08| 1957.47|
|2021-07-30 23:46:38| 2531.75|
|2021-04-27 15:44:22| 2667.93|
|2021-01-23 19:12:33| 1392.45|
|2021-02-22 01:01:30| 1778.07|
|2021-07-05 22:49:37| 2322.63|
|2021-05-02 06:39:23| 2951.98|
+-------------------+--------+
only showing top 20 rows



In [8]:
dollar_df.printSchema()

root
 |-- txn_date: timestamp (nullable = true)
 |-- usd_rate: string (nullable = true)



In [9]:
joined_df=df1.join(dollar_df,df1['txn_date']==dollar_df['txn_date'],'left').drop(dollar_df['txn_date'])
joined_df.show()

+-----------+--------+---------+--------------------+--------------------+------------------+------------+-------------------+--------+
|        NFT|token_id|num_sales|            username|       owner_address|       payment_amt|payment_type|           txn_date|usd_rate|
+-----------+--------+---------+--------------------+--------------------+------------------+------------+-------------------+--------+
|CryptoPunks|    9998|        4|0xef764bac8a438e7...|0xef764bac8a438e7...|124457.06752488602|         ETH|2021-10-28 23:50:37|  4417.4|
|CryptoPunks|    7804|        2|                null|0xf4b4a58974524e1...|            4200.0|         ETH|2021-03-11 00:48:39| 1827.06|
|CryptoPunks|    3100|        2|                null|0x7b8961f67382c51...|            4200.0|         ETH|2021-03-11 20:57:01| 1767.77|
|CryptoPunks|    4156|        5|         TheReal4156|0x4c1194386e12156...|            2500.0|         ETH|2021-12-09 20:38:17| 3902.49|
|CryptoPunks|    5217|        2|----------------

In [10]:
joined_df.printSchema()

root
 |-- NFT: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- num_sales: long (nullable = true)
 |-- username: string (nullable = true)
 |-- owner_address: string (nullable = true)
 |-- payment_amt: double (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- txn_date: timestamp (nullable = true)
 |-- usd_rate: string (nullable = true)



In [11]:

#uses "col('usd_price)" to order since cannot use joined_df['usd_price'] because that column does not exist in joined_df
update_df=joined_df.withColumn('usd_price', (joined_df['payment_amt']*joined_df['usd_rate'])).orderBy(col('usd_price').desc())

final_df=update_df.select(
        update_df['NFT'],\
        update_df['token_id'],\
        update_df['num_sales'],\
        update_df['username'],\
        update_df['owner_address'],\
        update_df['txn_date'],\
        format_number(update_df['payment_amt'],2).alias('payment_amt'),\
        update_df['payment_type'],\
        update_df['usd_rate'],\
        format_number(update_df['usd_price'],2).alias('usd_price'),\
)
final_df.show(truncate=False)

+-----------+--------+---------+------------------------------------------+------------------------------------------+-------------------+-----------+------------+--------+--------------+
|NFT        |token_id|num_sales|username                                  |owner_address                             |txn_date           |payment_amt|payment_type|usd_rate|usd_price     |
+-----------+--------+---------+------------------------------------------+------------------------------------------+-------------------+-----------+------------+--------+--------------+
|CryptoPunks|9998    |4        |0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db|0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db|2021-10-28 23:50:37|124,457.07 |ETH         |4417.4  |549,776,650.08|
|CryptoPunks|4156    |5        |TheReal4156                               |0x4c1194386e121566b79d1f1dac9ef282348f04f3|2021-12-09 20:38:17|2,500.00   |ETH         |3902.49 |9,756,225.00  |
|CryptoPunks|7804    |2        |null                        

In [12]:
final_df.printSchema()

root
 |-- NFT: string (nullable = true)
 |-- token_id: string (nullable = true)
 |-- num_sales: long (nullable = true)
 |-- username: string (nullable = true)
 |-- owner_address: string (nullable = true)
 |-- txn_date: timestamp (nullable = true)
 |-- payment_amt: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- usd_rate: string (nullable = true)
 |-- usd_price: string (nullable = true)



Etherscan API Ethereum and Token Data

In [14]:
owner_add_df=df1.select(df1['owner_address'],df1['txn_date'])
owner_add_df.show()

+--------------------+-------------------+
|       owner_address|           txn_date|
+--------------------+-------------------+
|0xef764bac8a438e7...|2021-10-28 23:50:37|
|0xf4b4a58974524e1...|2021-03-11 00:48:39|
|0x7b8961f67382c51...|2021-03-11 20:57:01|
|0x4c1194386e12156...|2021-12-09 20:38:17|
|0xe301612b0751f6d...|2021-07-30 21:24:47|
|0xdf37ac79f8e9b69...|2021-09-11 18:41:23|
|0xd6a984153acb6c9...|2021-07-30 18:36:56|
|0x0a8f4037729accb...|2021-08-24 06:34:02|
|0x7cd3fc0e4ef9589...|2021-08-06 18:18:56|
|0xd2c1a0138716f0f...|2021-09-04 21:39:01|
|0x999e77c988c4c14...|2021-08-28 14:24:55|
|0xce90a7949bb7889...|2021-07-30 23:45:46|
|0x2e675eeae4747c2...|2021-07-31 14:11:48|
|0x1251122f1d77fa4...|2021-02-19 14:27:08|
|0xce90a7949bb7889...|2021-07-30 23:46:38|
|0xe301612b0751f6d...|2021-04-27 15:44:22|
|0x266892ed0d40ea5...|2021-01-23 19:12:33|
|0x4d8e16a70f38414...|2021-02-22 01:01:30|
|0x93321b43d594aff...|2021-07-05 22:49:37|
|0xf3016b7608acfbe...|2021-05-02 06:39:23|
+----------

In [15]:
def eth_balance(eth_address):
    etherscan_url='https://api.etherscan.io/api?'
    eth_api_key='HIVRQXJ7RICD1E2J3U1IYQ74888AI1FWEV'
    param={'ETH_balance':{'module':'account','action':'balance','address':eth_address,'tag':'latest','apikey':eth_api_key}}
    response=requests.get(etherscan_url,params=param['ETH_balance'])
    response_dict=response.json()
    return response_dict['result']

eth_udf=udf(lambda x : eth_balance(x))
eth_balance_df=owner_add_df.withColumn('eth_balance',eth_udf(owner_add_df['owner_address'])).cache()


In [16]:
#getting ETH balances and dropping duplicates
eth_balance_df1=eth_balance_df.select(
    eth_balance_df['owner_address'],\
    eth_balance_df['txn_date'],\
    format_number((eth_balance_df['eth_balance']/10**18),2).alias('eth_balance')
).dropDuplicates(['owner_address'])
eth_balance_df1.show()

+--------------------+-------------------+-----------+
|       owner_address|           txn_date|eth_balance|
+--------------------+-------------------+-----------+
|0x040da2c46493300...|2021-09-29 01:47:03|       0.35|
|0x04ae2f0bda04f14...|2021-03-13 19:15:28|       0.19|
|0x06e63138f3241a4...|2021-09-15 07:22:48|       0.58|
|0x0a8f4037729accb...|2021-08-24 06:34:02|       0.00|
|0x1251122f1d77fa4...|2021-02-19 14:27:08|       0.00|
|0x1db12c2a7c80356...|2021-10-23 02:26:26|     119.05|
|0x22eab1c78521596...|2021-08-28 17:51:27|      10.31|
|0x266892ed0d40ea5...|2021-01-23 19:12:33|       0.00|
|0x2e336836cf0365e...|2021-09-25 15:53:39|      61.80|
|0x2e675eeae4747c2...|2021-07-31 14:11:48|       0.55|
|0x34669322bdfca9e...|2021-11-08 23:00:02|     270.97|
|0x3b93cbfb99560ab...|2021-07-29 21:20:25|      46.75|
|0x43fb99c5bc88c9c...|2021-05-18 05:17:32|       0.13|
|0x4c1194386e12156...|2021-12-09 20:38:17|       0.44|
|0x4d8e16a70f38414...|2021-02-22 01:01:30|       1.28|
|0x52d9651

In [17]:

def token_balance(eth_address):
    etherscan_url='https://api.etherscan.io/api?'
    eth_api_key='HIVRQXJ7RICD1E2J3U1IYQ74888AI1FWEV'
    token_dict={
    'Wrapped_eth':'0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2',
    'Tether':'0xdac17f958d2ee523a2206206994597c13d831ec7',
    'Usdc':'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
    'Dai':'0x6b175474e89094c44da98b954eedeac495271d0f',}
    new_dict={}
    for i in token_dict.keys():
        param={'token_balance':{'module':'account','action':'tokenbalance','contractaddress':token_dict[i],'address':eth_address,'tag':'latest','apikey':eth_api_key}}
        response=requests.get(etherscan_url,params=param['token_balance'])
        response_dict=response.json()
        new_dict[i]=response_dict['result']
    return new_dict


token_udf=udf(lambda x : token_balance(x),MapType(StringType(),StringType()))
token_balance_df=owner_add_df.withColumn('token_balance',token_udf(owner_add_df['owner_address'])).cache()




In [18]:
token_balance_df.show()


+--------------------+-------------------+--------------------+
|       owner_address|           txn_date|       token_balance|
+--------------------+-------------------+--------------------+
|0xef764bac8a438e7...|2021-10-28 23:50:37|{Usdc -> 83000000...|
|0xf4b4a58974524e1...|2021-03-11 00:48:39|{Usdc -> 0, Dai -...|
|0x7b8961f67382c51...|2021-03-11 20:57:01|{Usdc -> 0, Dai -...|
|0x4c1194386e12156...|2021-12-09 20:38:17|{Usdc -> 0, Dai -...|
|0xe301612b0751f6d...|2021-07-30 21:24:47|{Usdc -> 37561321...|
|0xdf37ac79f8e9b69...|2021-09-11 18:41:23|{Usdc -> 0, Dai -...|
|0xd6a984153acb6c9...|2021-07-30 18:36:56|{Usdc -> 0, Dai -...|
|0x0a8f4037729accb...|2021-08-24 06:34:02|{Usdc -> 0, Dai -...|
|0x7cd3fc0e4ef9589...|2021-08-06 18:18:56|{Usdc -> 0, Dai -...|
|0xd2c1a0138716f0f...|2021-09-04 21:39:01|{Usdc -> 0, Dai -...|
|0x999e77c988c4c14...|2021-08-28 14:24:55|{Usdc -> 0, Dai -...|
|0xce90a7949bb7889...|2021-07-30 23:45:46|{Usdc -> 53542315...|
|0x2e675eeae4747c2...|2021-07-31 14:11:4

In [19]:
token_balance_df.printSchema()

root
 |-- owner_address: string (nullable = true)
 |-- txn_date: timestamp (nullable = true)
 |-- token_balance: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)



In [20]:

token_balance_df1=token_balance_df.select(token_balance_df['owner_address'],\
                                        token_balance_df['txn_date'],\
                                        token_balance_df['token_balance'],\
                                        ).dropDuplicates(['owner_address'])
token_balance_df1.show()

+--------------------+-------------------+--------------------+
|       owner_address|           txn_date|       token_balance|
+--------------------+-------------------+--------------------+
|0x040da2c46493300...|2021-09-29 01:47:03|{Usdc -> 0, Dai -...|
|0x04ae2f0bda04f14...|2021-03-13 19:15:28|{Usdc -> 0, Dai -...|
|0x06e63138f3241a4...|2021-09-15 07:22:48|{Usdc -> 0, Dai -...|
|0x0a8f4037729accb...|2021-08-24 06:34:02|{Usdc -> 0, Dai -...|
|0x1251122f1d77fa4...|2021-02-19 14:27:08|{Usdc -> 0, Dai -...|
|0x1db12c2a7c80356...|2021-10-23 02:26:26|{Usdc -> 0, Dai -...|
|0x22eab1c78521596...|2021-08-28 17:51:27|{Usdc -> 0, Dai -...|
|0x266892ed0d40ea5...|2021-01-23 19:12:33|{Usdc -> 0, Dai -...|
|0x2e336836cf0365e...|2021-09-25 15:53:39|{Usdc -> 10010233...|
|0x2e675eeae4747c2...|2021-07-31 14:11:48|{Usdc -> 0, Dai -...|
|0x34669322bdfca9e...|2021-11-08 23:00:02|{Usdc -> 0, Dai -...|
|0x3b93cbfb99560ab...|2021-07-29 21:20:25|{Usdc -> 0, Dai -...|
|0x43fb99c5bc88c9c...|2021-05-18 05:17:3

In [21]:
import pyspark.sql.functions as F
from pyspark.sql.types import MapType, ArrayType, DataType, TimestampType, FloatType


token_balance_df2=token_balance_df1.select(\
    token_balance_df1['owner_address'],\
    token_balance_df1['txn_date'],\
    format_number(token_balance_df1['token_balance']['Wrapped_eth'].cast(FloatType())/10**18,2).alias('Wrapped_ETH'),\
    format_number(token_balance_df1['token_balance']['Dai'].cast(FloatType())/10**18,2).alias('DAI'),\
    format_number(token_balance_df1['token_balance']['Usdc'].cast(FloatType())/10**6,2).alias('USDC'),\
    format_number(token_balance_df1['token_balance']['Tether'].cast(FloatType())/10**6,2).alias('Tether'),\
    )


token_balance_df2.show()


+--------------------+-------------------+-----------+----------+------------+------------+
|       owner_address|           txn_date|Wrapped_ETH|       DAI|        USDC|      Tether|
+--------------------+-------------------+-----------+----------+------------+------------+
|0x040da2c46493300...|2021-09-29 01:47:03|       0.00|      0.00|        0.00|        0.00|
|0x04ae2f0bda04f14...|2021-03-13 19:15:28|       0.00|      0.00|        0.00|        0.00|
|0x06e63138f3241a4...|2021-09-15 07:22:48|       0.00|      0.00|        0.00|        0.00|
|0x0a8f4037729accb...|2021-08-24 06:34:02|       0.00|      0.00|        0.00|        0.00|
|0x1251122f1d77fa4...|2021-02-19 14:27:08|       0.00|      0.00|        0.00|        0.00|
|0x1db12c2a7c80356...|2021-10-23 02:26:26|       0.00|      0.00|        0.00|        0.00|
|0x22eab1c78521596...|2021-08-28 17:51:27|       0.00|      0.00|        0.00|  208,952.52|
|0x266892ed0d40ea5...|2021-01-23 19:12:33|       0.00|      0.00|        0.00|  